Import Required Packages

In [256]:
#|export
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import psutil
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import wd_classes
from wd_classes import FormEntry
import importlib
importlib.reload(wd_classes)
from wd_classes import FormEntry
from openpyxl import load_workbook
import tkinter as tk
import traceback
from fuzzywuzzy import fuzz


In [222]:
#|export
#Change these paths according to your system:
chrome_profile_path = "C:\\Users\\Joshu\OneDrive\\Documents\\GitHub\\WorkdayAutomatedJobApp\\ChromeProfile"
chrome_driver = "C:/Users/Joshu/AppData/Local/Microsoft/WindowsApps/chromedriver.exe"
initial_job_url = "https://accenture.wd3.myworkdayjobs.com/AccentureCareers/login?redirect=%2FAccentureCareers%2Fjob%2FNJ---Florham-Park%2FActimize-Developer---4624221_10912829-2%2Fapply"
#"https://jci.wd5.myworkdayjobs.com/JCI/login?redirect=%2FJCI%2Fjob%2FMissoula-Montana-United-States-of-America%2FHVAC-Truck-Base-Mechanic--Union-_WD30167702%2Fapply%3Fsource%3DCareers_Section"

your_email = "JoshuaLGodwin@outlook.com"
designated_password = "aTestPass@1234"
resume_path = "C:\\Users\\Joshu\\OneDrive\\Documents\\Career\\Resume\\Joshua Godwin Resume - Manufacturing.docx"

login_info_database_path = "login_info.csv"

job_questionare_answers = "Answers.xlsx"

In [195]:
#open the chrome window
import win32com.shell.shell as shell
commands = f'chrome.exe --remote-debugging-port=9222 --user-data-dir="{chrome_profile_path}"'
shell.ShellExecuteEx(lpVerb='runas', lpFile='cmd.exe', lpParameters='/c '+commands)

{'hInstApp': 42, 'hProcess': <PyHANDLE:0>}

Get current open chrome browser

In [196]:
#|export
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

#set driver to already open chrome window
driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\2876514780.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)


Error Handling

In [223]:
from logging_config import get_logger
#importlib.reload(logging_config)

logger = get_logger("jupyter_notebook")
logger.debug("This is a debug message from the Jupyter notebook")
logger.info("This is an info message from the Jupyter notebook")
"""
# Add a file handler
fh = logging.FileHandler('logs.log')
fh.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt="%Y-%m-%d %H:%M:%S")
fh.setFormatter(formatter)
logger.addHandler(fh)
"""

'\n# Add a file handler\nfh = logging.FileHandler(\'logs.log\')\nfh.setLevel(logging.DEBUG)\nformatter = logging.Formatter(\'%(asctime)s - %(name)s - %(levelname)s - %(message)s\', datefmt="%Y-%m-%d %H:%M:%S")\nfh.setFormatter(formatter)\nlogger.addHandler(fh)\n'

In [224]:
class stuck_on_login_page(Exception):
    print(f'page did not move past login page')
    pass

class stuck_on_create_account(Exception):
    print(f'Account not created')
    pass

class invalid_form_entry_type(Exception):
    print(f'answer type is not valid')

page did not move past login page
Account not created
answer type is not valid


Functions

In [225]:
#|export
#get login info saved in csv database
def wd_get_login_info(company_name,database=login_info_database_path):
    #open csv database
    df = pd.read_csv(database,dtype={'Password': str})
    
    #get email and password
    email = df.loc[df["Company"] == company_name, 'Email']
    email = email.values[0]
    password = df.loc[df["Company"] == company_name, 'Password']
    password = password.values[0]

    return email, password

In [226]:
#|export
#get login info saved in csv database
def write_login_info(company_name,email,password,database=login_info_database_path):
    #open csv database
    df = pd.read_csv(database)

    # add a new row to the DataFrame with the new person's name and company name
    new_row = pd.DataFrame({'Company': [f'{company_name}'], 'Email': [f'{email}'], 'Password': [f'{password}']})

    # concatenate the new DataFrame with the existing DataFrame
    df = pd.concat([df, new_row], ignore_index=False)

    # write the updated DataFrame to the existing CSV file
    df.to_csv(database, index=False)

In [227]:
#|export
#check if this url has a workday account in database
def wd_check_account_exists(company_name, login_info=login_info_database_path):
    df = pd.read_csv(login_info,dtype={'Password': str})
    if company_name in df["Company"].values:
        return True
    else:
        return False

In [228]:
#|export
def check_page_type(page_type):
    #check to see if on page = "page_type"
    try:
        page_title_obj = driver.find_element_by_xpath(f'//h2[text()="{page_type}"]')
    except:
        return False
    #page_title_obj = driver.find_element_by_id("authViewTitle")
    page_title = page_title_obj.text
    print(page_title)
    if page_title == page_type:
        return True
    else:
        return False

In [229]:
#|export
#create new wd account
def wd_create_account(company_name, username, password):
    #get form items
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element_by_css_selector("input[data-automation-id='password']")
    verify_password_text_box = driver.find_element_by_css_selector("input[data-automation-id='verifyPassword']")
    terms_checkbox = driver.find_element_by_css_selector("input[data-automation-id='createAccountCheckbox']")

    #fill in form
    email_text_box.clear()
    email_text_box.send_keys(your_email)

    #for some reason clear doesn't work with this field, so alternative is highlighting the text to clear
    password_text_box.send_keys(Keys.CONTROL,"a", Keys.DELETE)
    password_text_box.send_keys(password)
    password_text_box.send_keys(Keys.TAB)

    #See if password was accepted
    try:
        driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        error_msg = driver.find_element_by_css_selector("p[data-automation-id='inputError']")
        if error_msg.is_displayed():
            print("Passsword has error")
        else:
            print("password valid, but inputerror label exists")

    except NoSuchElementException:
        print("Password is valid")

    verify_password_text_box.clear()
    verify_password_text_box.send_keys(password)

    #check terms and conditions checkbox
    if terms_checkbox.get_attribute("checked") == "true":
        print('was already checked')
    else:
        wait = WebDriverWait(driver,5)
        terms_checkbox.click()

    #Get create account button
    create_account_button_filter = driver.find_element_by_xpath("//button[@data-automation-id='createAccountSubmitButton']/preceding-sibling::div")
    create_account_button = driver.find_element_by_xpath("//button[@data-automation-id='createAccountSubmitButton']")

    #hover over click filter
    hover = ActionChains(driver).move_to_element(create_account_button_filter)
    hover.perform()
    
    actions = ActionChains(driver)
    actions.move_to_element(create_account_button).click().perform()
    time.sleep(3)

    #Add to database if account created successfully
    if check_page_type('Create Account'):
        logger.info(f"div preceding create account button labeled: {create_account_button.get_attribute('aria-label')}")
        raise stuck_on_create_account(f"Did not move on to next page from create account")
    else:
        write_login_info(company_name, username, password)

In [230]:
#|export
def wd_sign_in(email, password):
    # Get email and password elements
    email_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'Email Address')]/following::input[@type='text'][1]")
    password_text_box = driver.find_element(By.XPATH, "//*[contains(text(), 'password')]/following::input[@type='password'][1]")
    
    #Fill in email and password
    email_text_box.send_keys(email)
    password_text_box.send_keys(password)

    #wait for a moment
    driver.implicitly_wait(2)
    
    #Get sign in button and click
    from selenium.webdriver.common.action_chains import ActionChains
    sign_in_button_filter = driver.find_element_by_xpath("//button[contains(text(), 'Sign In')]/preceding-sibling::div")
    #driver.execute_script("arguments[0].style.display='none'", sign_in_button_filter)
    sign_in_button = driver.find_element_by_xpath("//button[contains(text(), 'Sign In')]")
    
    sign_in_button_filter.click()

    #hover over click filter
    hover = ActionChains(driver).move_to_element(sign_in_button_filter)
    hover.perform()

    actions = ActionChains(driver)
    actions.move_to_element(sign_in_button).click().perform()
    time.sleep(3)

    #check to see if still on same page
    if check_page_type('Sign In'):
        raise stuck_on_login_page("Stuck at login page")
    else:
        print("no errors found on login page")

In [231]:
#|export
def get_current_company():
    #get url and pull the text after the first / after myworkdayjobs.com
    current_url = driver.current_url

    # Split the URL by "/"
    parts = current_url.split("/")

    # Get the text following ".com/" and before the next slash
    company_name = parts[3]

    return company_name

In [232]:
#|export
def wd_login_page(new_email,new_password):
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.ID, "input-4")))
    
    current_company = get_current_company()

    #check if account exists for this site and login if has account or create account and then login
    if wd_check_account_exists(current_company) is True:
        email, password = wd_get_login_info(current_company)
        wd_sign_in(email, password)
    else:
        create_account_button = driver.find_element_by_xpath("//button[contains(text(), 'Create Account')]")
        create_account_button.click()
        
        wd_create_account(current_company, new_email, new_password)

In [233]:
#|export
def wd_startYourApplication_page():
    
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-automation-id='applyManually']")))

    manual_apply_btn = driver.find_element_by_css_selector("a[data-automation-id='applyManually']")
    manual_apply_btn.click()

In [234]:
#|export

def fuzzy_match(df, question):
    max_ratio = -1
    index = -1
    match_floor = 60
    for i, row in df.iterrows():
        ratio = fuzz.token_sort_ratio(row['Questions'], question)

        if ratio > max_ratio:
            max_ratio = ratio
            index = i
    return index, max_ratio

def get_answer(df,question):
    
    #pull answer from form
    try:
        index, similarity = fuzzy_match(df, question)
        if similarity < 60:
            logger.info(f"Similarity too low, could not find match for question {question}, set answer to blank")
            answer = ""
            return answer
        answer: str
        answer = df.at[index, 'Answers']
        logger.info(f"answer found for question: {question}, answer is: {answer}")
        
    except:
        logger.info(f"Couldn't get answer for {question}, set to blank answer")
        answer = ""

    if pd.isna(answer):
        logger.info(f"Answer in database for {question} is NaN, set answer to blank")
        answer = ""

    return answer

In [235]:
df = pd.read_excel(job_questionare_answers, sheet_name="Answers", dtype=str)

answer = get_answer(df, 'Country / Territory Phone Code')
print(answer)

United States of America (+1)


In [236]:
#|export
def wait_data_automation_id_loaded(dataautomationid,t=20):
    try:
        wait = WebDriverWait(driver, t)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
        time.sleep(1)
    except Exception as e:
        print("start of wait data failure")
        traceback.print_exc()
        print(str(e))
        raise Exception("wait for loading failed")


In [237]:
def fill_sub_section(driver, df, exp_container):
    wait = WebDriverWait(driver, 2)
    # locate all input elements in work experience 1
    inputs_list = exp_container.find_elements_by_css_selector("[id^='input-']")

    #remove any elements that don't have their id ending in a number
    new_inputs_list = []
    for i in range(len(inputs_list)):
        current_id = inputs_list[i].get_attribute('ID')
        if current_id[-1].isdigit() != False:
            new_inputs_list.append(inputs_list[i])
    inputs_list = new_inputs_list

    #move through each form entry and fill in
    entries = []
    for element in inputs_list:
        #make sure element still exists
        try:
            if len(driver.find_elements_by_xpath(f"//*[@id='{element.get_attribute('ID')}']")) > 0:
                pass
            else:
                logger.info(f"Could not find element: {element}, this element was most likely an element that existed prior to an update to one of the previous entries")
                continue
        except Exception as e:
            logger.error(f"Checking if element in input list failed")
            continue
        
        #create element object
        entry = FormEntry(element)
        
        #get the type of answer form it is
        entry.get_id()
        entry.get_tag()
        entry.get_answer_type()
        entry.get_label()

        #skip if a span element
        if entry.tag == 'span':
            continue
        
        #pull answer from form
        answer = get_answer(df,entry.label)
        entry.add_answer(answer)

        #update to make sure element is current
        entry.update_element(driver) 

        # Get window height
        window_height = driver.execute_script("return window.innerHeight;")
        
        viewport_size = driver.execute_script("return {width: window.innerWidth, height: window.innerHeight};")
        size = entry.element.size

        if element.location_once_scrolled_into_view['y'] >= 0:
            logger.info(f"Element: {entry}, is already in view")
            #logger.info(f"Viewport size: {viewport_size['width']} x {viewport_size['height']}")
            #logger.info(f"size of element {entry}: size: {size['width']} x {size['height']}")
            #actions = ActionChains(driver)
            #actions.move_to_element_with_offset(entry.element,0,0.5).perform()
            
            #parent_element = entry.element.find_element_by_xpath(f"./preceding-sibling")
            #driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().bottom - window.innerHeight / 2);", parent_element)
            driver.execute_script("arguments[0].scrollIntoView(true);", entry.element)
                
        else:
            #driver.execute_script("window.scrollBy(0, arguments[0].getBoundingClientRect().top- window.innerHeight / 2);", entry.element)
            driver.execute_script("arguments[0].scrollIntoView(true);", entry.element)    
            
            logger.info(f"Viewport size: {viewport_size['width']} x {viewport_size['height']}")
            logger.info(f"scrolled into view for element: {entry}, size of element Element size: {size['width']} x {size['height']}")
        
        #fill in answer
        #wait.until(EC.element_to_be_clickable((By.XPATH, f"//*[@id='{entry.ID}']")))
        entry.write_answer(driver)
            
        #append entry to entries list
        entries.append(entry)
        

In [238]:
#|export
def delete_sub_section(aria_label):
    delete_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,{aria_label})]")
    logger.info(f"Found {len(delete_btn)} delete buttons starting with aria-label: {aria_label}")
    tries = 0
    max_tries = 10
    while len(delete_btn) > 0:
        try:
            delete_btn_id = delete_btn[0].get_attribute('aria-label')
        except:
            break
        logger.info(f"delete button: {aria_label} found")
        delete_btn[0].click()
        logger.info(f"clicked on button: {str(delete_btn_id)}")
        tries += 1
        delete_btn = driver.find_elements_by_xpath(f"//button[starts-with(@aria-label,{aria_label})]")
        if tries >= max_tries:
            break
    else:
        logger.info(f"No buttons found starting with aria-label: {aria_label}")

def add_sub_section(form_container_id,positioning='following-sibling'):
    add_btn = driver.find_elements_by_xpath(f"//*[@data-automation-id='{form_container_id}']/{positioning}::div//button[starts-with(@aria-label,'Add')]")
    add_btn[0].click()

In [239]:
#|export
def fill_section(browser_driver, container_auto_id, start_of_sheets_name, numb_sheets, qa_database, implicit_wait_time):
    
    #wait until page has loaded form
    if numb_sheets == 1:
        try:
            wait_data_automation_id_loaded(f"{container_auto_id}{1}")
        except:
            wait_data_automation_id_loaded(container_auto_id)
    else:
        wait_data_automation_id_loaded(f"{container_auto_id}{1}")

    #get question/answer database and sheets
    sheet_names = pd.ExcelFile(qa_database).sheet_names
    sheets = [name for name in sheet_names if name.startswith(start_of_sheets_name)]
    required_sheets = list(sheets)

    #fill in each form for each job
    for i in range(numb_sheets):

        #put sheet required to fill in form as the active dataframe
        df = pd.read_excel(qa_database,required_sheets[i],dtype=str)
        
        #get the container element
        if numb_sheets == 1:
            try:
                form_container = driver.find_element_by_xpath(f"//div[@data-automation-id='{container_auto_id}']")
            except:
                form_container = driver.find_element_by_xpath(f"//div[@data-automation-id='{container_auto_id}{i+1}']")
        else:
            form_container = driver.find_element_by_xpath(f"//div[@data-automation-id='{container_auto_id}{(i+1)}']")
        
        #fill in work experience section
        fill_sub_section(driver,df,form_container)

        #click on add another so long as not last work experience to add
        if i+1 != numb_sheets and numb_sheets > 1:  
            #click work experience button
            form_container_id = form_container.get_attribute('data-automation-id')
            add_sub_section(form_container_id, 'following-sibling')


In [240]:
#|export
def upload_file(resume_path):
    # Locate the upload button using its ID or CSS selector
    upload_button = driver.find_element_by_xpath(f"//input[@data-automation-id='file-upload-input-ref']")
    upload_button.send_keys(resume_path)


In [241]:
#Open job link, sign in, and start manual application
def start_job_app(url, email, password):
    #get job posting url
    driver.get(url)

    #login or create account and log in
    wd_login_page(email, password)

    #Click on start manual application
    try:
        wd_startYourApplication_page()
    except:
        logger.warning(f"Apply manually button was not pressed. This could be because the job app had already been started or an error")



In [242]:
def click_element(element_type, handle_type, value):
    try:
        wait = WebDriverWait(driver, 5)
        wait.until(EC.element_to_be_clickable((By.XPATH,f"//{element_type}[@{handle_type}='{value}']"))).click()
        logger.info(f"succesfully clicked on {element_type} with handle_type = {handle_type}, and value = {value}")
    except Exception as e:
        logger.error(f"could not clicked with {element_type} with handle_type = {handle_type}, and value = {value}")

In [243]:
#|export
#
#click the save and continue button
def go_to_next():
    click_element('button','data-automation-id','bottom-navigation-next-button')


In [244]:
def fill_all_forms(job_questionare_answers,debug_index = 0):
    global status_message
    global i

    #hide the pesky header that gets in way of interacting with objects
    header = driver.find_element_by_tag_name("header")
    driver.execute_script("arguments[0].style.display='none'", header)
    
    #get number of sheets for work experience and education
    sheet_names = pd.ExcelFile(job_questionare_answers).sheet_names
    work_sheets = [name for name in sheet_names if name.startswith("WorkExp")]
    ed_sheets = [name for name in sheet_names if name.startswith("Education")]
    
    
    numb_work_sheets = len(work_sheets)
    numb_ed_sheets = len(ed_sheets)
    logger.info(f'Number of work_sheets: {numb_work_sheets}, nubmer of education sheets; {numb_ed_sheets}')

    implicit_wait_time = 0.2
    
    #list of functions ot run in order
    functions = [
            (fill_section, (driver, 'contactInformationPage', 'Answers', 1, job_questionare_answers, implicit_wait_time)),
            (go_to_next, ()),
            (wait_data_automation_id_loaded, ('workExperienceSection',)),
            (delete_sub_section, ("'Delete Work Experience'",)),
            (fill_section, (driver,'workExperience-','WorkExp',numb_work_sheets, job_questionare_answers,implicit_wait_time)),
            (delete_sub_section, ("'Delete Education'",)),
            (add_sub_section, ('educationSection', 'descendant')),
            (fill_section, (driver,'education-','Education',numb_ed_sheets,job_questionare_answers,implicit_wait_time)),
            (upload_file, (resume_path,)),
            (go_to_next, ()),
            (fill_section, (driver,'primaryQuestionnairePage','Answers',1,job_questionare_answers,implicit_wait_time)),
            (go_to_next, ()),
            (fill_section, (driver,'voluntaryDisclosuresPage','RacistQuestions',1,job_questionare_answers,implicit_wait_time)),
            (go_to_next, ()),
            (fill_section, (driver,'selfIdentificationPage','Disabilities',1,job_questionare_answers,implicit_wait_time)),
            (go_to_next, ())
        ]

    #figure out page on and start from that index
    page_header = driver.find_elements_by_xpath(f"//h2[@tabindex='-1']")
    header_names = []
    for ele in page_header:
        header_names.append(ele.text)
    
    logger.info(f"header names are: {header_names}")
    
    #set index based on page|
    if "My Information" in header_names:
        i = 0
    elif "My Experience" in header_names:
        i = 2
    elif "Application Questions" in header_names:
        i = 11

    if debug_index == 0:
        pass
    else:
        i = debug_index

    #run functions starting from index i
    while i < len(functions) and i > -2:
        func, args = functions[i]
        try:
            func(*args)
            i += 1
        except Exception as e:
            i = i + 1
            traceback.print_exc()
            logger.warning(f"Error in function {func.__name__}, args: {args},  error exception: {str(e)}")
            status_message.set(f"Warning, could not complete function: {func.__name__}, correct errors and then press continue")
            break


In [258]:
fill_all_forms(job_questionare_answers,5)


start of wait data failure
Message: 



Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\3029300324.py", line 5, in wait_data_automation_id_loaded
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
  File "c:\ProgramData\Anaconda3\envs\machinelearning\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 

Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\1805981572.py", line 66, in fill_all_forms
    func(*args)
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\124282166.py", line 34, in fill_section
    fill_sub_section(driver,df,form_container)
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\799928598.py", line 74, in fill_sub_section
    entry.write_answer(driver)
  File "c:\Users\Joshu\OneDrive\Documents\GitHub\WorkdayAutomatedJob

In [247]:
#Create Simple Interface for Script

root = tk.Tk()
root.title("Simple GUI")

#Default Login Info
email = tk.StringVar()
email.set(your_email)

password = tk.StringVar()
password.set(designated_password)

# Create a frame to group the components
group_frame = tk.Frame(root)
group_frame.pack(padx=10, pady=10)

#Input Url Label
link_label = tk.Label(group_frame, text='Input Job Application Url')
link_label.grid(row=0, column=0, columnspan=3, padx=2,pady=2, sticky='w')

url = tk.StringVar()
url.set(initial_job_url)

status_message = tk.StringVar()
status_message.set("")

#Input Url Box
link_entry = tk.Entry(group_frame, width=50, textvariable=url)
link_entry.grid(row=1, column=0, columnspan=3, padx=2, pady=10, sticky='w')

def start_job_app_click():
    url = link_entry.get()
    email = email_entry.get()
    password = password_entry.get()
    start_job_app(url, email, password)

def fill_all_forms_tk_command():
    fill_all_forms(job_questionare_answers)

# Start Application button
start_button = tk.Button(group_frame, text="Start Application", command=start_job_app_click)
start_button.grid(row=2, column=0, padx=2, pady=2, sticky='w')

# Continue button
i = 0
continue_button = tk.Button(group_frame, text="Continue", command=fill_all_forms_tk_command)
continue_button.grid(row=2, column=1, padx=2, pady=2)

# Output label
error_label = tk.Label(group_frame, textvariable=status_message)
error_label.grid(row=3, column=0, columnspan=3, padx=2, pady=2)

#Login Entries
email_label = tk.Label(group_frame, text="Email")
email_label.grid(row=4, column=0, columnspan=2, padx=2, pady=2, sticky="w")
email_entry = tk.Entry(group_frame, textvariable=email, width= 30)
email_entry.grid(row=5, column=0,columnspan=2, padx=10, pady=10, sticky="w")

password_label = tk.Label(group_frame, text="Password")
password_label.grid(row=4, column=2, columnspan=2, padx=2, pady=2, sticky="w")
password_entry = tk.Entry(group_frame, textvariable=password, width=30)
password_entry.grid(row=5, column=2, columnspan=2, padx=10, pady=10, sticky="w")

root.mainloop()

no errors found on login page
made it to here
start of wait data failure
Message: 



Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\3029300324.py", line 5, in wait_data_automation_id_loaded
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
  File "c:\ProgramData\Anaconda3\envs\machinelearning\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 

Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\1805981572.py", line 66, in fill_all_forms
    func(*args)
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\124282166.py", line 34, in fill_section
    fill_sub_section(driver,df,form_container)
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\799928598.py", line 74, in fill_sub_section
    entry.write_answer(driver)
  File "c:\Users\Joshu\OneDrive\Documents\GitHub\WorkdayAutomatedJob

start of wait data failure
Message: 



Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\3029300324.py", line 5, in wait_data_automation_id_loaded
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
  File "c:\ProgramData\Anaconda3\envs\machinelearning\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



start of wait data failure
Message: 



Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\3029300324.py", line 5, in wait_data_automation_id_loaded
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"[data-automation-id='{dataautomationid}']")))
  File "c:\ProgramData\Anaconda3\envs\machinelearning\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\124282166.py", line 7, in fill_section
    wait_data_automation_id_loaded(f"{container_auto_id}{1}")
  File "C:\Users\Joshu\AppData\Local\Temp\ipykernel_13780\3029300324.py", line 11, in wait_data_automation_id_loaded
    raise Exception("wait for loading failed")
Exception: wait for loading failed

During handling of the above excep

In [150]:
ele = driver.find_element_by_xpath(f"//div[@data-uxi-widget-type='popup']/descendant::div[normalize-space(text())=\"Bachelor's Degree\"]")
ele.click()

In [ ]:
"""
To do:
Figure out way to scroll into view of element so that click element doesn't constantly fail if out of view
make all text matching not be case sensitive
Add fuzzy matching so that this can work with other job applications where questions are similar but not exact
Improve UI and add retry button
"""